# GroupKFold (5 splits) vs Leave-One-Out

**Problem**: CV-LB intercept (0.0525) > Target (0.0347). Even CV=0 would give LB=0.0525.

**Hypothesis**: The mixall kernel uses GroupKFold (5 splits) instead of Leave-One-Out.
This is a different validation scheme that may have different CV-LB characteristics.

**Base model**: exp_030 GP+MLP+LGBM (CV 0.008298, LB 0.0877)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

# Original Leave-One-Out splits (for comparison)
def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# GroupKFold splits (mixall approach)
def generate_groupkfold_splits(X, Y, n_splits=5):
    """Generate Group K-Fold splits across the solvents."""
    groups = X["SOLVENT NAME"]
    n_groups = len(groups.unique())
    n_splits = min(n_splits, n_groups)
    
    gkf = GroupKFold(n_splits=n_splits)
    
    for train_idx, test_idx in gkf.split(X, Y, groups):
        yield (
            (X.iloc[train_idx], Y.iloc[train_idx]),
            (X.iloc[test_idx], Y.iloc[test_idx]),
        )

def generate_groupkfold_ramp_splits(X, Y, n_splits=5):
    """Generate Group K-Fold splits across the solvent ramps."""
    groups = X["SOLVENT A NAME"].astype(str) + "_" + X["SOLVENT B NAME"].astype(str)
    n_groups = len(groups.unique())
    n_splits = min(n_splits, n_groups)
    
    gkf = GroupKFold(n_splits=n_splits)
    
    for train_idx, test_idx in gkf.split(X, Y, groups):
        yield (
            (X.iloc[train_idx], Y.iloc[train_idx]),
            (X.iloc[test_idx], Y.iloc[test_idx]),
        )

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension (for GP): {SimpleFeaturizer().feats_dim}')

Simple feature dimension (for GP): 18


In [6]:
# MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32], output_dim=3, dropout=0.1):
        super().__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

print('MLP model defined')

MLP model defined


In [7]:
# GP+MLP+LGBM Ensemble Model (same as exp_030)
class GPMLPLGBMEnsemble:
    """GP+MLP+LGBM ensemble model."""
    
    def __init__(self, data='single', gp_weight=0.3, mlp_weight=0.4, lgbm_weight=0.3):
        self.data = data
        self.mixed = (data == 'full')
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        
    def train_model(self, X, Y):
        Y_np = Y.values if hasattr(Y, 'values') else Y
        
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        # Scale
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        # Train GP (on simple features)
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        self.gp_models = []
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, random_state=42)
            gp.fit(X_simple_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        X_torch = torch.tensor(X_full_scaled, dtype=torch.double)
        Y_torch = torch.tensor(Y_np, dtype=torch.double)
        
        self.mlp = MLP(input_dim=X_full_scaled.shape[1]).double().to(device)
        optimizer = torch.optim.Adam(self.mlp.parameters(), lr=5e-4, weight_decay=1e-5)
        criterion = nn.HuberLoss()
        
        dataset = TensorDataset(X_torch, Y_torch)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        self.mlp.train()
        for epoch in range(200):
            for xb, yb in loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                pred = self.mlp(xb)
                loss = criterion(pred, yb)
                loss.backward()
                optimizer.step()
        
        # Train LGBM
        self.lgbm_models = []
        for i in range(3):
            model = lgb.LGBMRegressor(
                n_estimators=500, learning_rate=0.03, max_depth=6,
                num_leaves=31, reg_alpha=0.1, reg_lambda=0.1,
                random_state=42, verbose=-1
            )
            model.fit(X_full_scaled, Y_np[:, i])
            self.lgbm_models.append(model)
        
    def predict(self, X):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        # Scale
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions
        gp_preds = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gp_models])
        
        # MLP predictions
        self.mlp.eval()
        with torch.no_grad():
            X_torch = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            mlp_preds = self.mlp(X_torch).cpu().numpy()
        
        # LGBM predictions
        lgbm_preds = np.column_stack([m.predict(X_full_scaled) for m in self.lgbm_models])
        
        # Ensemble
        final_preds = (self.gp_weight * gp_preds + 
                       self.mlp_weight * mlp_preds + 
                       self.lgbm_weight * lgbm_preds)
        
        # Clip to valid range [0, 1]
        final_preds = np.clip(final_preds, 0, 1)
        
        return torch.tensor(final_preds, dtype=torch.double)

print('GPMLPLGBMEnsemble defined')

GPMLPLGBMEnsemble defined


In [8]:
# Compare Leave-One-Out vs GroupKFold for single solvent data
print('Comparing Leave-One-Out vs GroupKFold for single solvent data...')

X, Y = load_data("single_solvent")
print(f'Single solvent data: {X.shape[0]} samples, {len(X["SOLVENT NAME"].unique())} solvents')

# Leave-One-Out
print('\n--- Leave-One-Out (24 folds) ---')
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions_loo = []
all_actuals_loo = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GPMLPLGBMEnsemble(data='single')
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    all_predictions_loo.append(predictions_np)
    all_actuals_loo.append(test_Y.values)

all_predictions_loo = np.vstack(all_predictions_loo)
all_actuals_loo = np.vstack(all_actuals_loo)
mse_loo = np.mean((all_predictions_loo - all_actuals_loo) ** 2)
print(f'Leave-One-Out MSE: {mse_loo:.6f}')

Comparing Leave-One-Out vs GroupKFold for single solvent data...
Single solvent data: 656 samples, 24 solvents

--- Leave-One-Out (24 folds) ---


0it [00:00, ?it/s]

1it [00:20, 20.73s/it]

2it [00:41, 20.62s/it]

3it [01:00, 19.93s/it]

...

22it [08:17, 23.35s/it]

23it [08:36, 21.92s/it]

24it [08:57, 21.82s/it]

24it [08:57, 22.40s/it]

Leave-One-Out MSE: 0.008560


In [ ]:
# GroupKFold (5 splits)
print('\n--- GroupKFold (5 folds) ---')
split_generator = generate_groupkfold_splits(X, Y, n_splits=5)
all_predictions_gkf = []
all_actuals_gkf = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GPMLPLGBMEnsemble(data='single')
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    all_predictions_gkf.append(predictions_np)
    all_actuals_gkf.append(test_Y.values)

all_predictions_gkf = np.vstack(all_predictions_gkf)
all_actuals_gkf = np.vstack(all_actuals_gkf)
mse_gkf = np.mean((all_predictions_gkf - all_actuals_gkf) ** 2)
print(f'GroupKFold (5) MSE: {mse_gkf:.6f}')

In [ ]:
# Summary
print('\n=== SUMMARY ===')
print(f'Leave-One-Out (24 folds) MSE: {mse_loo:.6f}')
print(f'GroupKFold (5 folds) MSE: {mse_gkf:.6f}')
print(f'\nBaseline (exp_030): Single Solvent MSE 0.007943')

if mse_gkf < mse_loo:
    print(f'\nGroupKFold is {(mse_loo - mse_gkf) / mse_loo * 100:.2f}% better than Leave-One-Out')
else:
    print(f'\nLeave-One-Out is {(mse_gkf - mse_loo) / mse_gkf * 100:.2f}% better than GroupKFold')